In [19]:
# install various non-standard packages
%pip install -q scipy
%pip install -q librosa

# just FYI list all packages+libs known in this file. Some of these are installed in the environment.yml file
#import pkg_resources
#for m in pkg_resources.working_set:
#    print(f"{m.project_name}=={m.version}")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [18]:
# Using the demon function
from demon import square_law
import numpy

sigma= 500
mu=20000
x= mu + sigma * numpy.random.randn(2, 4) # 2x4 array gaussian distrib

# from demon
print("x=",x)
print("result=",square_law(x))

x= [[20072.53290346 20842.47517501 20453.89616616 19911.49228247]
 [19529.33552191 20884.45473968 20519.80229633 19876.45354371]]
100
result= [[ 0.20860465]
 [-0.20860465]]
